In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [2]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
# ## Plotly
# import plotly.offline as py
# import plotly.graph_objs as go
# py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

Using TensorFlow backend.


In [304]:
ids = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/id.Dennis+Schwartz.txt", sep="[\r\n]+", header=None, names=["id"])
label_3class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.3class.Dennis+Schwartz.txt", sep="[\r\n]+", names=["3class_label"])
label_4class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.4class.Dennis+Schwartz.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/rating.Dennis+Schwartz.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/subj.Dennis+Schwartz.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support 

In [307]:
dennis = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)
dennis = dennis.sample(frac=1)

(1027, 5)

In [290]:
dennis["strongly neg"]=dennis.apply(lambda x:x["4class_label"]==0, axis=1)
dennis["neg"]=dennis.apply(lambda x:x["4class_label"]==1, axis=1)
dennis["pos"]=dennis.apply(lambda x:x["4class_label"]==2, axis=1)
dennis["strongly pos"]=dennis.apply(lambda x:x["4class_label"]==3, axis=1)

In [291]:
y_d = dennis['3class_label']
dennis['n']=dennis.apply(lambda x:x["3class_label"]==0, axis=1)
dennis["neg"]=dennis.apply(lambda x:x["3class_label"]==1, axis=1)
dennis["pos"]=dennis.apply(lambda x:x["3class_label"]==2, axis=1)
y_d_encode = dennis[['n', 'neg', 'pos']]
x_d = list(dennis['subj_extraction'])
x_d_padded = gd.padded_doc(x_d)

In [292]:
from sklearn.model_selection import train_test_split
xtrain_d_padded, xvalid_d_padded, ytrain_d, yvalid_d = train_test_split(x_d_padded, y_d, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [293]:
xtrain_d_padded, xvalid_d_padded, ytrain_d_encode, yvalid_d_encode = train_test_split(x_d_padded, y_d_encode,
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [302]:
xtrain_d_padded.shape
x_d_padded.shape

(1027, 3000)

In [272]:
import glove_dl23 as gd

In [296]:
vocab_size, embedding_matrix, padded_docs = gd.get_glove_m(total_text)

In [297]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.1529    , -0.24279   ,  0.89837003, ..., -0.59100002,
         1.00390005,  0.20664001],
       ..., 
       [-0.86329001,  0.63463998, -0.33840999, ..., -0.052484  ,
         0.20648   ,  0.46869999],
       [ 0.19193999,  0.38505   , -0.22404   , ..., -0.16169   ,
         0.080152  ,  0.29284999],
       [-0.045712  ,  0.14199001, -0.30963001, ..., -0.22722   ,
         0.76585001,  0.66973001]])

## cnn 

In [299]:

from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras.models import Sequential, Model
def build_model(embedding_matrix, vocab_size):
    # define model
    embedding_dim = 50
    filter_sizes = (2, 4)
    num_filters = 10
    dropout_prob = (0.5, 0.8)
    hidden_dims = 50

    # Build model
    sequence_length = 3000
    input_shape = (sequence_length,)

    model_input = Input(shape=input_shape)

    # Static model does not have embedding layer

    z = Embedding(vocab_size, 100, input_length=sequence_length,weights=[embedding_matrix], name="embedding")(model_input)
    z.trainable = False
    
    z = Dropout(dropout_prob[0])(z)

    # Convolutional block
    conv_blocks = []
    for sz in filter_sizes:
        conv = Convolution1D(filters=num_filters,
                             kernel_size=sz,
                             padding="valid",
                             activation="relu",
                             strides=1)(z)
        conv = MaxPooling1D(pool_size=1)(conv)
        conv = Flatten()(conv)
        conv_blocks.append(conv)
    z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

    z = Dropout(dropout_prob[1])(z)
    z = Dense(hidden_dims, activation="relu")(z)
    model_output = Dense(3, activation="sigmoid")(z)

    model = Model(model_input, model_output)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    weights = np.array([v for v in embedding_matrix])
    print("Initializing embedding layer with glove weights, shape", weights.shape)
    embedding_layer = model.get_layer("embedding")
    embedding_layer.set_weights([weights])
    return model

# # Initialize weights with word2vec
# if model_type == "CNN-non-static":
#     weights = np.array([v for v in embedding_weights.values()])
#     print("Initializing embedding layer with word2vec weights, shape", weights.shape)
#     embedding_layer = model.get_layer("embedding")
#     embedding_layer.set_weights([weights])

# Train the model
model = build_model(embedding_matrix, vocab_size)
model.fit(xtrain_d_padded, ytrain_d_encode, epochs=10, verbose=1)


Initializing embedding layer with glove weights, shape (41441, 100)
Epoch 1/10
924/924 [==============================] - 21s 23ms/step - loss: 1.0877 - acc: 0.3983
Epoch 2/10
924/924 [==============================] - 19s 21ms/step - loss: 1.0867 - acc: 0.4177
Epoch 3/10
924/924 [==============================] - 19s 21ms/step - loss: 1.0761 - acc: 0.4113
Epoch 4/10
924/924 [==============================] - 19s 21ms/step - loss: 1.0756 - acc: 0.4123
Epoch 5/10
924/924 [==============================] - 19s 21ms/step - loss: 1.0578 - acc: 0.4145
Epoch 6/10
924/924 [==============================] - 20s 21ms/step - loss: 1.0721 - acc: 0.4242
Epoch 7/10
924/924 [==============================] - 19s 21ms/step - loss: 1.0365 - acc: 0.4297
Epoch 8/10
924/924 [==============================] - 20s 21ms/step - loss: 1.0330 - acc: 0.4318
Epoch 9/10
924/924 [==============================] - 19s 21ms/step - loss: 1.0222 - acc: 0.4426
Epoch 10/10
924/924 [==============================] - 19s 

In [300]:
model.evaluate(xvalid_d_padded, yvalid_d_encode)

103/103 [==============================] - 1s 13ms/step


[1.0518304422063736, 0.40776699115929094]

In [23]:
model.predict(xtrain_d_padded)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 400, 100)     1791100     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 400, 100)     0           embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 399, 10)      2010        dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (

## visuliaze the prediction

In [75]:
# evaluate the model
loss, accuracy = model.evaluate(xvalid_d_padded_docs, yvalid_d_encode, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 41.747573


In [27]:
xtrain_d_glv = np.array(xtrain_d_glv)
xvalid_d_glv = np.array(xvalid_d_glv)
xtrain_d_glv.shape

(924, 300)

In [318]:
# serialize model to JSON
model_json = model.to_json()
with open("model_gl_cnn.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_gl_cnn.h5")
print("Saved model to disk")

Saved model to disk


# glove_nn on the other writer

In [248]:
ids = pd.read_csv("scale_data/scaledata/James+Berardinelli/id.James+Berardinelli.txt", sep="[\r\n]+", header=None, names=["id"])
# label_3class = pd.read_csv("scale_data/scaledata/James+Berardinelli/label.3class.James+Berardinelli.txt", sep="[\r\n]+", names=["3class_label"])
label_3class = pd.read_csv("scale_data/scaledata/James+Berardinelli/label.3class.James+Berardinelli.txt", sep="[\r\n]+",names=["3class_label"])

label_4class = pd.read_csv("scale_data/scaledata/James+Berardinelli/label.4class.James+Berardinelli.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/James+Berardinelli/rating.James+Berardinelli.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/James+Berardinelli/subj.James+Berardinelli.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarnin

In [263]:
james = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)


In [308]:

james['n']=james.apply(lambda x:x["3class_label"]==0, axis=1)
james["neg"]=james.apply(lambda x:x["3class_label"]==1, axis=1)
james["pos"]=james.apply(lambda x:x["3class_label"]==2, axis=1)
y_j_encode = james[['n', 'neg', 'pos']]

In [309]:
x_j = james['subj_extraction']
y_j = james['3class_label']
x_j_padded = gd.padded_doc(x_j)

In [310]:
from sklearn.model_selection import train_test_split
xtrain_j_padded, xvalid_j_padded, ytrain_j, yvalid_j = train_test_split(x_j_padded, y_j, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [311]:
print (x_j_padded.shape)
print (y_j.shape)

(1307, 3000)
(1307,)


In [312]:
from sklearn.model_selection import train_test_split
xtrain_j_padded, xvalid_j_padded, ytrain_j_encode, yvalid_j_encode = train_test_split(x_j_padded, y_j_encode, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [313]:
print (x_j_padded.shape)
print (y_j_encode.shape)
print (xvalid_j_padded.shape)
print (yvalid_j_encode.shape)

(1307, 3000)
(1307, 3)
(131, 3000)
(131, 3)


In [314]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os

## load the pre_trained model and evaluate on james directly

In [319]:
# load json and create model
json_file = open('model_gl_cnn.json', 'r')
loaded_model_cnn_json = json_file.read()
json_file.close()
loaded_model_cnn = model_from_json(loaded_model_cnn_json)
# load weights into new model
loaded_model_cnn.load_weights("model_gl_cnn.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model_cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
loaded_model_cnn.evaluate(xvalid_j_padded, yvalid_j_encode, verbose=1)
# print("%s: %.2f%%" % (loaded_model_cnn.metrics_names[1], score[1]*100))
# # loaded_model.summary()
# score

Loaded model from disk
131/131 [==============================] - 2s 18ms/step


[1.732673660489439, 0.33078880187209325]

In [200]:
loaded_model_cnn.predict(xvalid_j_padded)
# print("%s: %.2f%%" % (loaded_model_cnn.metrics_names[1], score[1]*100))


[0.82195321937183763, 0.54212456122859498]

## fine tuning freeze the E layer.

In [89]:
# for layer in loaded_model_cnn.layers[:2]:
#     layer.trainable = False

In [205]:
loaded_model_cnn

In [206]:
from keras import optimizers

loaded_model_cnn.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [271]:
model_j = build_model(embedding_matrix, vocab_size)
model_j.fit(xtrain_j_padded, ytrain_j_encode, epochs = 50, verbose = 1)

Initializing embedding layer with glove weights, shape (41441, 100)


ValueError: Error when checking input: expected input_15 to have shape (100,) but got array with shape (3000,)

In [228]:
model_j.evaluate(xvalid_j_padded, yvalid_j_encode, verbose=1)
# model_j.evaluate(xtrain_j_padded, ytrain_j_encode, verbose=1)
# model.metrics_names


91/91 [==============================] - 0s 264us/step


[1.1063710202227581, 0.29670329506580645]

In [229]:
model_j.predict(xvalid_j_padded)

array([[ 0.64245242,  0.50412488,  0.23868002],
       [ 0.49703661,  0.5076555 ,  0.32914522],
       [ 0.43550697,  0.48260731,  0.56188565],
       [ 0.28396404,  0.61782485,  0.48895967],
       [ 0.42587209,  0.48146808,  0.46928751],
       [ 0.31395876,  0.61788809,  0.60919958],
       [ 0.42675418,  0.4544012 ,  0.44078216],
       [ 0.33290228,  0.41679204,  0.58824396],
       [ 0.56185287,  0.46434608,  0.2707774 ],
       [ 0.49795598,  0.41308469,  0.51998979],
       [ 0.20698871,  0.66136622,  0.50882316],
       [ 0.51466602,  0.50216717,  0.27823126],
       [ 0.44028118,  0.55967009,  0.35109553],
       [ 0.26097831,  0.52796692,  0.65655339],
       [ 0.3712368 ,  0.62871826,  0.40809503],
       [ 0.38964769,  0.52673215,  0.42186907],
       [ 0.58997673,  0.38191703,  0.43637866],
       [ 0.40669954,  0.510086  ,  0.44548333],
       [ 0.32908437,  0.50886804,  0.58529431],
       [ 0.19547509,  0.39568335,  0.64250672],
       [ 0.37727225,  0.48087683,  0.377

In [90]:
loaded_model_cnn.fit(xtrain_j_padded, ytrain_j_encode, epochs=10, verbose=1)

Epoch 1/10
  64/1176 [>.............................] - ETA: 2s - loss: 0.7982 - acc: 0.5104

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/training.py:953: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1176/1176 [==============================] - 2s 2ms/step - loss: 0.7868 - acc: 0.5105
Epoch 2/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7680 - acc: 0.5162
Epoch 3/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7587 - acc: 0.5156
Epoch 4/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7346 - acc: 0.5334
Epoch 5/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7385 - acc: 0.5357
Epoch 6/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7319 - acc: 0.5266
Epoch 7/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7068 - acc: 0.5468
Epoch 8/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7124 - acc: 0.5454
Epoch 9/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7007 - acc: 0.5496
Epoch 10/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.6894 - acc: 0.5683


## the third writer

In [91]:
ids = pd.read_csv("scale_data/scaledata/Scott+Renshaw/id.Scott+Renshaw.txt", sep="[\r\n]+", header=None, names=["id"])
label_3class = pd.read_csv("scale_data/scaledata/Scott+Renshaw/label.3class.Scott+Renshaw.txt", sep="[\r\n]+", names=["3class_label"])
label_4class = pd.read_csv("scale_data/scaledata/Scott+Renshaw/label.4class.Scott+Renshaw.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/Scott+Renshaw/rating.Scott+Renshaw.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/Scott+Renshaw/subj.Scott+Renshaw.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support 

In [133]:
scott = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)
# y_s is a column with 3 values: 0,1,2
y_s = scott['3class_label']
scott['n']=scott.apply(lambda x:x["3class_label"]==0, axis=1)
scott["neg"]=scott.apply(lambda x:x["3class_label"]==1, axis=1)
scott["pos"]=scott.apply(lambda x:x["3class_label"]==2, axis=1)
# y_s_encode are three columns with binary values for each column
y_s_encode = scott[['n', 'neg', 'pos']]
x_s = scott['subj_extraction']
x_s_padded = gd.padded_doc(x_s)

In [134]:
xtrain_s, xvalid_s, ytrain_s, yvalid_s = train_test_split(x_s_padded, y_s,  
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=False)

In [135]:
xtrain_s, xvalid_s, ytrain_s_encode, yvalid_s_encode = train_test_split(x_s_padded, y_s_encode, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=False)

## the fourth writer

In [98]:
ids = pd.read_csv("scale_data/scaledata/Steve+Rhodes/id.Steve+Rhodes.txt", sep="[\r\n]+", header=None, names=["id"])
label_3class = pd.read_csv("scale_data/scaledata/Steve+Rhodes/label.3class.Steve+Rhodes.txt", sep="[\r\n]+", names=["3class_label"])
label_4class = pd.read_csv("scale_data/scaledata/Steve+Rhodes/label.4class.Steve+Rhodes.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/Steve+Rhodes/rating.Steve+Rhodes.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/Steve+Rhodes/subj.Steve+Rhodes.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support 

In [117]:
Rhodes = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)
# y_s is a column with 3 values: 0,1,2
y_r = scott['3class_label']
Rhodes['n']=Rhodes.apply(lambda x:x["3class_label"]==0, axis=1)
Rhodes["neg"]=Rhodes.apply(lambda x:x["3class_label"]==1, axis=1)
Rhodes["pos"]=Rhodes.apply(lambda x:x["3class_label"]==2, axis=1)
# y_s_encode are three columns with binary values for each column
y_r_encode = Rhodes[['n', 'neg', 'pos']]
x_r = Rhodes['subj_extraction']
x_r_padded = gd.padded_doc(x_r)

In [127]:
xtrain_r_padded, xvalid_r_padded, ytrain_r, yvalid_r = train_test_split(x_r_padded, y_r,  
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=False)

In [129]:
xtrain_r_padded, xvalid_r_padded, ytrain_r_encode, yvalid_r_encode = train_test_split(x_r_padded, y_r_encode, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=False)

### preparation: total text for embedding matrix

In [106]:
# total text
x_d = list(x_d)
x_j = list(x_j)
x_s = list(x_s)
x_r = list(x_r)
total_text = x_d+x_j+x_s+x_r

In [107]:
# get embedding_matrix for total_text
vocab_size, embedding_matrix, padded_docs = gd.get_glove_m(total_text)

### preparation: get padded text 

## Train the model from the first writer

In [143]:
Model_cnn = build_model(embedding_matrix, vocab_size)

Initializing embedding layer with glove weights, shape (41441, 100)


In [144]:
Model_cnn.fit(xtrain_d_padded, ytrain_d_encode, epochs=10, verbose=1)

Epoch 1/10
924/924 [==============================] - 5s 5ms/step - loss: 1.0485 - acc: 0.3355
Epoch 2/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9433 - acc: 0.3550
Epoch 3/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9428 - acc: 0.3983
Epoch 4/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9284 - acc: 0.4091
Epoch 5/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9241 - acc: 0.4253
Epoch 6/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9233 - acc: 0.4405
Epoch 7/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9256 - acc: 0.4275
Epoch 8/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9182 - acc: 0.4318
Epoch 9/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9134 - acc: 0.4318
Epoch 10/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9119 - acc: 0.4318


In [145]:
# serialize model to JSON
model_cnn_json = model.to_json()
with open("model_gl_cnn.json", "w") as json_file:
    json_file.write(model_cnn_json)
# serialize weights to HDF5
model.save_weights("model_gl_cnn.h5")
print("Saved model to disk")

Saved model to disk


## train on the second (pretrained on B)

In [149]:
Model_cnn.fit(xtrain_j_padded, ytrain_j_encode, epochs=10, verbose=1)

Epoch 1/10
811/811 [==============================] - 2s 3ms/step - loss: 1.0767 - acc: 0.3822
Epoch 2/10
811/811 [==============================] - 2s 3ms/step - loss: 1.0709 - acc: 0.3822
Epoch 3/10
811/811 [==============================] - 2s 3ms/step - loss: 1.0714 - acc: 0.3798
Epoch 4/10
811/811 [==============================] - 2s 3ms/step - loss: 1.0808 - acc: 0.3810
Epoch 5/10
811/811 [==============================] - 2s 3ms/step - loss: 1.0612 - acc: 0.3847
Epoch 6/10
811/811 [==============================] - 2s 3ms/step - loss: 1.0583 - acc: 0.3859
Epoch 7/10
811/811 [==============================] - 2s 3ms/step - loss: 1.0680 - acc: 0.3896
Epoch 8/10
811/811 [==============================] - 3s 3ms/step - loss: 1.0662 - acc: 0.3933
Epoch 9/10
811/811 [==============================] - 3s 3ms/step - loss: 1.0477 - acc: 0.3933
Epoch 10/10
811/811 [==============================] - 2s 3ms/step - loss: 1.0436 - acc: 0.3872


In [155]:
Model_cnn.evaluate(xvalid_j_padded, yvalid_j_encode, verbose=1)
print("%s: %.2f%%" % (Model_cnn.metrics_names[1], score[1]*100))
Model_cnn.predict(xvalid_j_padded)

91/91 [==============================] - 0s 247us/step
acc: 54.21%


array([[ 0.81836802,  0.93762517,  0.77988225],
       [ 0.68789679,  0.93742245,  0.75056595],
       [ 0.71980613,  0.9147976 ,  0.77256364],
       [ 0.68642354,  0.91774011,  0.78740674],
       [ 0.69518661,  0.90007198,  0.70963788],
       [ 0.71924949,  0.90312862,  0.78281647],
       [ 0.64705807,  0.94560784,  0.83421904],
       [ 0.59793162,  0.88889885,  0.74650306],
       [ 0.57446182,  0.88032877,  0.78512013],
       [ 0.68418664,  0.91499037,  0.78228426],
       [ 0.75535387,  0.9332456 ,  0.81973702],
       [ 0.74912798,  0.93696374,  0.72559249],
       [ 0.67369902,  0.93870443,  0.80707961],
       [ 0.61071694,  0.93996388,  0.80251342],
       [ 0.71017903,  0.91614455,  0.80288166],
       [ 0.64273942,  0.93425745,  0.79099518],
       [ 0.67749566,  0.90729791,  0.77032697],
       [ 0.59798819,  0.91707909,  0.80967766],
       [ 0.69291252,  0.9386313 ,  0.78256845],
       [ 0.72203147,  0.95351833,  0.78164041],
       [ 0.69630164,  0.90479159,  0.813

In [148]:
# load json and create model
json_file = open('model_gl_cnn.json', 'r')
loaded_model_cnn_json = json_file.read()
json_file.close()
loaded_model_cnn = model_from_json(loaded_model_cnn_json)
# load weights into new model
loaded_model_cnn.load_weights("model_gl_cnn.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model_cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model_cnn.evaluate(xvalid_j_padded, yvalid_j_encode, verbose=1)
print("%s: %.2f%%" % (loaded_model_cnn.metrics_names[1], score[1]*100))
# loaded_model.summary()

Loaded model from disk
91/91 [==============================] - 1s 8ms/step
acc: 54.21%
